# 2.0 Join data

AIMS

En este archivo se realiza la union de los datos para formar un unico dataset

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from mypackage import dir
from mypackage.transforms import convert_variables
from mypackage.calculates import proportion_categories, df_correlations#, calculate_time_between_purchases
from rfm import calculate_rfm
from help import calculate_time_between_purchases

modality =  modality = 'p'
project = 'australian'
data = dir.make_dir_line(modality, project) 
processed = data('processed')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

## 2.1 Load data

In [3]:
df_customers = pd.read_parquet(processed/'df_clientes.parquet.gzip')
df_customers = df_customers.loc[:,['id_cliente', 'sexo', 'age_days']]
df_customers = convert_variables(df_customers, ['id_cliente', 'sexo'], 'str')
df_customers.head()

,id_cliente,sexo,age_days
0,1001,1,34247
1,1002,1,30466
2,1003,0,39356
3,1004,0,14272
4,1005,0,2794


In [4]:
df_products = pd.read_parquet(processed/'df_productos.parquet.gzip')
df_products = df_products.loc[:,['id_producto', 'category', 'valor', 'valor_produccion']]
df_products.head()

,id_producto,category,valor,valor_produccion
0,759ne,Short,40.0,18.400
1,759bl,Short,40.0,18.400
2,759az,Short,40.0,18.400
3,759ro,Short,40.0,18.400
4,761ro,Short,43.2,15.984


In [5]:
df_sales = pd.read_parquet(processed/'df_ventas.parquet.gzip')
df_sales.head()

,id_factura,fecha,id_cliente,id_producto,quantity,id_store,discount,per_discount2
0,1001,2019-03-16,1331,1634ra,1,2,1,20.0
1,1002,2019-03-16,1131,5821gr,1,1,0,0.0
2,1003,2019-03-16,1344,9151ro,1,2,1,5.0
3,1004,2019-03-16,1444,9145gr,2,2,1,5.0
4,1005,2019-03-16,1345,1367ve,2,2,0,0.0


In [6]:
print(df_sales.shape, df_products.shape)
df = df_sales.merge(df_products, on = ['id_producto'])
print(df.shape)

(9001, 8) (398, 4)
(9001, 11)


In [7]:
df['total_factura'] = df['valor'] * df['quantity']
df['total_produccion'] = df['valor_produccion'] * df['quantity']
df['total_descuento'] = (df['total_factura'] * df['per_discount2'])/100
df['total_factura_con_descuento'] = df['total_factura'] - df['total_descuento'] 
df['ganancia'] = df['total_factura_con_descuento'] - df['total_produccion'] 
df.head()

,id_factura,fecha,id_cliente,id_producto,quantity,id_store,discount,per_discount2,category,valor,valor_produccion,total_factura,total_produccion,total_descuento,total_factura_con_descuento,ganancia
0,1001,2019-03-16,1331,1634ra,1,2,1,20.0,Mini,44.8,14.336,44.8,14.336,8.96,35.84,21.504
1,1169,2019-04-16,1096,1634ra,5,1,0,0.0,Mini,44.8,14.336,224.0,71.680,0.00,224.00,152.320
2,2003,2019-09-16,1068,1634ra,1,1,0,0.0,Mini,44.8,14.336,44.8,14.336,0.00,44.80,30.464
3,2227,2019-11-04,1105,1634ra,2,0,0,0.0,Mini,44.8,14.336,89.6,28.672,0.00,89.60,60.928
4,2310,2019-11-22,1237,1634ra,2,2,1,5.0,Mini,44.8,14.336,89.6,28.672,4.48,85.12,56.448


## 2.2 Make some calculations

In [8]:
df_rfm = calculate_rfm(df, val_id_customer='id_cliente', 
                       val_id_facture = 'id_factura', 
                       val_money = 'ganancia', 
                       val_date = 'fecha')
df_rfm.head()

,id_cliente,recency,frequency,monetary
0,1001,82,21,894.448
1,1002,303,14,1127.408
2,1003,102,15,699.168
3,1004,86,19,1054.560
4,1005,183,22,1608.320


In [9]:
df_prop = proportion_categories(df, 
                                pet=0, 
                                val1='id_cliente', 
                                val2='category', 
                                val3='ganancia')
df_prop.head()

category,id_cliente,Babucha,Bermuda,Calza,Camisa,Capri,Chaleco,Jean,Mini,Musculosa,Pantalón,Pollera,Remera,Short,Strapless,Vestido
0,1001,0.0,0.000000,0.047619,0.142857,0.047619,0.047619,0.095238,0.095238,0.190476,0.0,0.000000,0.190476,0.142857,0.000000,0.0
1,1002,0.0,0.000000,0.000000,0.285714,0.071429,0.000000,0.071429,0.000000,0.142857,0.0,0.071429,0.357143,0.000000,0.000000,0.0
2,1003,0.0,0.000000,0.000000,0.333333,0.000000,0.066667,0.066667,0.000000,0.133333,0.0,0.000000,0.200000,0.200000,0.000000,0.0
3,1004,0.0,0.000000,0.000000,0.263158,0.052632,0.000000,0.157895,0.052632,0.315789,0.0,0.000000,0.157895,0.000000,0.000000,0.0
4,1005,0.0,0.045455,0.000000,0.272727,0.000000,0.000000,0.181818,0.045455,0.181818,0.0,0.000000,0.181818,0.045455,0.045455,0.0


In [10]:
df_disc = proportion_categories(df, 
                                pet=0, 
                                val1='id_cliente', 
                                val2='discount', 
                                val3='ganancia')
df_disc.rename(columns={'0':'local', '1':'no_local'}, inplace=True)
df_disc.head()

discount,id_cliente,local,no_local
0,1001,0.428571,0.571429
1,1002,0.500000,0.500000
2,1003,0.466667,0.533333
3,1004,0.578947,0.421053
4,1005,0.772727,0.227273


In [11]:
df_times = calculate_time_between_purchases(df, 
                                            customer_id_column = 'id_cliente',
                                            date_column = 'fecha'
                                            )
df_times = df_times.loc[:,['id_cliente', 'Time_Average']]
df_times.head()

,id_cliente,Time_Average
0,1001,86.0
1,1002,113.0
2,1003,117.0
3,1004,85.0
4,1005,76.0


## 2.3 Join

In [12]:
dataset = df_customers.copy()

In [13]:
print(dataset.shape, df_rfm.shape, df_prop.shape, df_disc.shape)
dataset = dataset.merge(df_rfm, on = ['id_cliente'])
print(dataset.shape)
dataset = dataset.merge(df_prop, on = ['id_cliente'])
print(dataset.shape)
dataset = dataset.merge(df_disc, on = ['id_cliente'])
print(dataset.shape)
dataset = dataset.merge(df_times, on = ['id_cliente'])
print(dataset.shape)

(500, 3) (500, 4) (500, 16) (500, 3)
(500, 6)
(500, 21)
(500, 23)
(500, 24)


In [14]:
dataset = dataset.loc[:,['id_cliente', 'sexo', 'age_days', 'recency', 'frequency', 'monetary',
       'Babucha', 'Bermuda', 'Calza', 'Camisa', 'Capri', 'Chaleco', 'Jean',
       'Mini', 'Pantalón', 'Pollera', 'Remera', 'Short',
       'Strapless', 'Vestido', 'local', 'Time_Average']]
dataset.head()

,id_cliente,sexo,age_days,recency,frequency,monetary,Babucha,Bermuda,Calza,Camisa,Capri,Chaleco,Jean,Mini,Pantalón,Pollera,Remera,Short,Strapless,Vestido,local,Time_Average
0,1001,1,34247,82,21,894.448,0.0,0.000000,0.047619,0.142857,0.047619,0.047619,0.095238,0.095238,0.0,0.000000,0.190476,0.142857,0.000000,0.0,0.428571,86.0
1,1002,1,30466,303,14,1127.408,0.0,0.000000,0.000000,0.285714,0.071429,0.000000,0.071429,0.000000,0.0,0.071429,0.357143,0.000000,0.000000,0.0,0.500000,113.0
2,1003,0,39356,102,15,699.168,0.0,0.000000,0.000000,0.333333,0.000000,0.066667,0.066667,0.000000,0.0,0.000000,0.200000,0.200000,0.000000,0.0,0.466667,117.0
3,1004,0,14272,86,19,1054.560,0.0,0.000000,0.000000,0.263158,0.052632,0.000000,0.157895,0.052632,0.0,0.000000,0.157895,0.000000,0.000000,0.0,0.578947,85.0
4,1005,0,2794,183,22,1608.320,0.0,0.045455,0.000000,0.272727,0.000000,0.000000,0.181818,0.045455,0.0,0.000000,0.181818,0.045455,0.045455,0.0,0.772727,76.0


## 2.4 Correlations

In [15]:
salida_correlations = df_correlations(dataset.loc[:, dataset.columns != 'id_sample'], sort_by_distance=True)
salida_correlations

,Column_1,Column_2,Correlation,type
59,Time_Average,frequency,-0.868681,Strong Negative
43,monetary,frequency,0.763058,Strong Positive
79,Time_Average,monetary,-0.687108,Moderate Negative
22,frequency,recency,-0.277476,Weak Negative
70,Jean,monetary,0.229744,Weak Positive
214,Remera,Jean,-0.210218,Weak Negative
151,Mini,Camisa,-0.202647,Weak Negative
234,Remera,Mini,-0.201129,Weak Negative
155,Short,Camisa,-0.184638,Weak Negative
295,Short,Remera,-0.173593,Weak Negative


In [16]:
# Save data
dataset.to_parquet(processed/'dataset.parquet.gzip', compression='gzip')

In [17]:
print('ok_')

ok_
